Protocols are a mechanism to achieve polymorphism in Elixir. Dispatching on a protocal is available to any data type as long as it implements the protocol.

In Elixir, we have two idioms for checking how many items there are in a data structure: `length` and `size`. `length` means the information must be computed. For example, `length(list)` needs to traverse the whole list to calculate its length. On the other head, `tuple_size(tuple)` and `byte_size(binary)` do not depend on the tuple and binary size as the size information is pre-computed in the data structure.

In [21]:
defprotocol Size do
  @doc "Calculate the size (and not the length!) of the data structure"
  def size(data)
end

{:module, Size, <<70, 79, 82, 49, 0, 0, 18, 148, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 2, 134, 0, 0, 0, 51, 11, 69, 108, 105, 120, 105, 114, 46, 83, 105, 122, 101, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, ...>>, {:__protocol__, 1}}

In [22]:
defimpl Size, for: BitString do
  def size(string), do: byte_size(string)
end

defimpl Size, for: Map do
  def size(map), do: map_size(map)
end

defimpl Size, for: Tuple do
  def size(tuple), do: tuple_size(tuple)
end

{:module, Size.Tuple, <<70, 79, 82, 49, 0, 0, 5, 204, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 194, 0, 0, 0, 20, 17, 69, 108, 105, 120, 105, 114, 46, 83, 105, 122, 101, 46, 84, 117, 112, 108, 101, 8, 95, 95, 105, 110, ...>>, {:__impl__, 1}}

In [23]:
 Size.size("foo")

3

In [24]:
Size.size({:ok, "hello"})

2

In [25]:
Size.size(%{label: "some label"})

1

In [26]:
Size.size(%{})

0

In [27]:
 Size.size([1, 2, 3])

Protocol.UndefinedError: 1

In [27]:
defmodule User do
  defstruct [:name, :age]
end

{:module, User, <<70, 79, 82, 49, 0, 0, 5, 156, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 181, 0, 0, 0, 18, 11, 69, 108, 105, 120, 105, 114, 46, 85, 115, 101, 114, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, ...>>, %User{age: nil, name: nil}}

In [28]:
defimpl Size, for: User do
  def size(_user), do: 2
end

{:module, Size.User, <<70, 79, 82, 49, 0, 0, 5, 152, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 181, 0, 0, 0, 19, 16, 69, 108, 105, 120, 105, 114, 46, 83, 105, 122, 101, 46, 85, 115, 101, 114, 8, 95, 95, 105, 110, 102, ...>>, {:__impl__, 1}}

In [29]:
defimpl Size, for: Any do
  def size(_), do: 0
end

{:module, Size.Any, <<70, 79, 82, 49, 0, 0, 5, 144, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 179, 0, 0, 0, 19, 15, 69, 108, 105, 120, 105, 114, 46, 83, 105, 122, 101, 46, 65, 110, 121, 8, 95, 95, 105, 110, 102, 111, ...>>, {:__impl__, 1}}

The implementation above is arguably not a reasonable one. For example, it makes no sense to say that the size of a PID or an Integer is 0.

In [30]:
defmodule OtherUser do
  @derive [Size]
  defstruct [:name, :age]
end

{:module, OtherUser, <<70, 79, 82, 49, 0, 0, 5, 172, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 186, 0, 0, 0, 18, 16, 69, 108, 105, 120, 105, 114, 46, 79, 116, 104, 101, 114, 85, 115, 101, 114, 8, 95, 95, 105, 110, 102, ...>>, %OtherUser{age: nil, name: nil}}

In [31]:
t = %OtherUser{name: 2, age: 3}

%OtherUser{age: 3, name: 2}

In [32]:
Size.size t

0

In [33]:
defmodule OtherUser2 do
  defstruct [:name, :age]
end

{:module, OtherUser2, <<70, 79, 82, 49, 0, 0, 5, 180, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 187, 0, 0, 0, 18, 17, 69, 108, 105, 120, 105, 114, 46, 79, 116, 104, 101, 114, 85, 115, 101, 114, 50, 8, 95, 95, 105, 110, ...>>, %OtherUser2{age: nil, name: nil}}

In [34]:
t2 = %OtherUser2{name: 2, age: 3}

%OtherUser2{age: 3, name: 2}

In [35]:
Size.size t2

Protocol.UndefinedError: 1

Elixir ships with some built-in protocols. In previous chapters, we have discussed the `Enum` module which provides many functions that work with any data structure that implements the `Enumerable` protocol,

Another useful example is the `String.Chars` protocol, which specifies how to convert a data structure with characters to a string. It's exposed via the `to_string` function.

In [35]:
tuple = {1, 2, 3}

{1, 2, 3}

In [36]:
"tuple: #{tuple}"

Protocol.UndefinedError: 1

In [36]:
"tuple: #{inspect tuple}"

"tuple: {1, 2, 3}"

The `Inspect` protocol is the protocol used to transform any data structure into a readable textual representation. That is what tools like IEx use to print results

whenever inspect value starts with `#`, it is representing a data structure in on-valid Elixir syntax. This means the inspect protocol is not reversible as information may be lost along the way

In [37]:
 inspect &(&1+2)

"#Function<6.127694169/1 in :erl_eval.expr/5>"